#Preprosessing data & TF - IDF

## Import Library

### Library yang digunakan

- **Pandas**

- **swifter**

- **PySastrawi**

- **scikit-learn**

- **networkx**

- **nltk**

- **numpy**

- **re**

In [4]:
import pandas as pd
import numpy as np
import string
import re #regrex libray
import nltk
import swifter
import Sastrawi
import networkx as nx

from nltk.tokenize import word_tokenize
from nltk.tokenize.punkt import PunktSentenceTokenizer
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.probability import FreqDist
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer

## Preprosessing Data

Ambil data abstrak yang sudah di ektraksi dari pta trunojoyo tadi.

In [5]:
df = pd.read_csv('hasil-crawl.csv')
df.head()

,judul_TA,abstrak
0,PENGARUH PERILAKU KONSUMEN TERHADAP KEPUTUSAN ...,"ABSTRAK\r\nHaryono Arifin, Pengaruh Perilaku K..."
1,PENGARUH DIMENSI KUALITAS PELAYANAN TERHADAP K...,ABSTRAK\r\n\r\nTujuan penelitian ini adalah un...
2,Analisis Inovasi Dan Keunggulan Bersaing \r\nD...,ABSTRAK\r\n\tTujuan penelitian ini adalah mend...
3,Pengaruh Retailing Mix terhadap Keputusan Pemb...,Tujuan penelitian ini adalah untuk mengetahui ...
4,ANALISIS TINGKAT RISIKO KREDIT \r\nPADA PD. BP...,Hasil dari penelitian ini dari perhitungan Cre...


Kemudian cek apakah ada missing value pada data yang di crawl.

In [6]:
df.isna().sum()

judul_TA    0
abstrak     1
dtype: int64

Jika ada missing value, hilangkan missing value nya dengan fungsi dropna.

In [7]:
df = df.dropna(axis=0, how='any')
df.isna().sum()

judul_TA    0
abstrak     0
dtype: int64

# Case Folding

Lalu lakukan case folding (mengganti semua data menjadi lowercase)terhadap data df.

In [8]:
# ------ Case Folding --------
# gunakan fungsi Series.str.lower() pada Pandas
df['abstrak'] = df['abstrak'].str.lower()


print('Case Folding Result : \n')
print(df['abstrak'].head(20))
print('\n\n\n')

Case Folding Result : 

0     abstrak\r\nharyono arifin, pengaruh perilaku k...
1     abstrak\r\n\r\ntujuan penelitian ini adalah un...
2     abstrak\r\n\ttujuan penelitian ini adalah mend...
3     tujuan penelitian ini adalah untuk mengetahui ...
4     hasil dari penelitian ini dari perhitungan cre...
5     abstrak\r\n\r\naththaariq, pengaruh kompetensi...
6     abstrak\r\n\tdharma abidin syah,kesimpulan: (1...
7     abstrak\r\n\r\npenelitian ini mengungkapkan be...
8     pln merupakan badan usaha milik negara yang be...
9     tujuan penelitian ini adalah untuk mengidentif...
10    pendekatan penelitian yang digunakan dalam pen...
11    diana agustini dewi, 2013. “analisis kinerja k...
12    tujuan penelitian ini adalah untuk mengetahui ...
13    muhammad sholeh, pengaruh iklan di televisi te...
14    tujuan dari penelitian ini adalah untuk menget...
15                                        abstrak\r\...
16    tujuan penelitian ini adalah untuk mengetahui ...
17    aplikasi nyata pem

# Tokenizing

Kemudian hilangkan karakter yang tidak termasuk datam ASCII, hilangkan angka, hilangkan simbol - simbol, dan juga spasi kosong.

Kemudian lakukan tokenizing pada data df tersbut.

In [9]:
import string 
import re #regex library

# import word_tokenize & FreqDist from NLTK
from nltk.tokenize import word_tokenize 
from nltk.probability import FreqDist

# ------ Tokenizing ---------

def remove_tweet_special(text):
    # remove tab, new line, ans back slice
    text = text.replace('\\t'," ").replace('\\n'," ").replace('\\u'," ").replace('\\',"")
    # remove non ASCII (emoticon, chinese word, .etc)
    text = text.encode('ascii', 'replace').decode('ascii')
    # remove mention, link, hashtag
    text = ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)"," ", text).split())
    # remove incomplete URL
    return text.replace("http://", " ").replace("https://", " ")
                
df['abstrak'] = df['abstrak'].apply(remove_tweet_special)

#remove number
def remove_number(text):
    return  re.sub(r"\d+", "", text)

df['abstrak'] = df['abstrak'].apply(remove_number)

#remove punctuation
def remove_punctuation(text):
    return text.translate(str.maketrans("","",string.punctuation))

df['abstrak'] = df['abstrak'].apply(remove_punctuation)

#remove whitespace leading & trailing
def remove_whitespace_LT(text):
    return text.strip()

df['abstrak'] = df['abstrak'].apply(remove_whitespace_LT)

#remove multiple whitespace into single whitespace
def remove_whitespace_multiple(text):
    return re.sub('\s+',' ',text)

df['abstrak'] = df['abstrak'].apply(remove_whitespace_multiple)

# remove single char
def remove_singl_char(text):
    return re.sub(r"\b[a-zA-Z]\b", "", text)

df['abstrak'] = df['abstrak'].apply(remove_singl_char)

# NLTK word rokenize 
def word_tokenize_wrapper(text):
    return word_tokenize(text)

df['abstrak_tokens'] = df['abstrak'].apply(word_tokenize_wrapper)

print('Tokenizing Result : \n') 
print(df['abstrak_tokens'].head(20))
print('\n\n\n')

Tokenizing Result : 

0     [abstrak, haryono, arifin, pengaruh, perilaku,...
1     [abstrak, tujuan, penelitian, ini, adalah, unt...
2     [abstrak, tujuan, penelitian, ini, adalah, men...
3     [tujuan, penelitian, ini, adalah, untuk, menge...
4     [hasil, dari, penelitian, ini, dari, perhitung...
5     [abstrak, aththaariq, pengaruh, kompetensi, do...
6     [abstrak, dharma, abidin, syahkesimpulan, terd...
7     [abstrak, penelitian, ini, mengungkapkan, bebe...
8     [pln, merupakan, badan, usaha, milik, negara, ...
9     [tujuan, penelitian, ini, adalah, untuk, mengi...
10    [pendekatan, penelitian, yang, digunakan, dala...
11    [diana, agustini, dewi, analisis, kinerja, keu...
12    [tujuan, penelitian, ini, adalah, untuk, menge...
13    [muhammad, sholeh, pengaruh, iklan, di, televi...
14    [tujuan, dari, penelitian, ini, adalah, untuk,...
15    [abstrak, satiyah, pengaruh, faktorfaktor, pel...
16    [tujuan, penelitian, ini, adalah, untuk, menge...
17    [aplikasi, nyata, pe

Setelah itu hitung frekuensi token yang sering muncul didalam dokumen dengan NTLK


In [10]:
# NLTK calc frequency distribution
def freqDist_wrapper(text):
    return FreqDist(text)

df['abstrak_tokens_fdist'] = df['abstrak_tokens'].apply(freqDist_wrapper)

print('Frequency Tokens : \n') 
print(df['abstrak_tokens_fdist'].head(20).apply(lambda x : x.most_common()))
ab = df['abstrak_tokens_fdist'].head(20).apply(lambda x : x.most_common())

Frequency Tokens : 

0     [(konsumen, 6), (pembelian, 6), (yang, 6), (ke...
1     [(konsumen, 7), (variabel, 7), (yang, 6), (kep...
2     [(inovasi, 5), (keunggulan, 5), (bersaing, 5),...
3     [(yang, 14), (variabel, 12), (dan, 8), (peneli...
4     [(sebesar, 9), (tahun, 7), (dari, 4), (pada, 4...
5     [(kompetensi, 14), (dosen, 8), (yang, 8), (var...
6     [(variabel, 16), (tipe, 10), (prestasi, 10), (...
7     [(biaya, 6), (kualitas, 6), (pada, 6), (ini, 5...
8     [(yang, 7), (pln, 5), (penelitian, 5), (kerja,...
9     [(menggunakan, 6), (persepsi, 6), (internet, 5...
10    [(ratio, 8), (yang, 6), (tidak, 6), (penelitia...
11    [(perusahaan, 11), (tahun, 11), (yang, 10), (a...
12    [(variabel, 9), (komitmen, 5), (terhadap, 5), ...
13    [(pesan, 9), (terhadap, 6), (pengaruh, 5), (un...
14    [(yang, 6), (dari, 5), (variabel, 5), (prestas...
15    [(dan, 20), (dengan, 13), (produktivitas, 9), ...
16    [(dimensi, 8), (penelitian, 5), (dan, 5), (ini...
17    [(dan, 8), (akademik,

# Filtering (Stopword Removal)

Setelah itu Lakukan filtring stopword dengan library ntlk

In [11]:
from nltk.corpus import stopwords

# ----------------------- get stopword from NLTK stopword -------------------------------
# get stopword indonesia
list_stopwords = stopwords.words('indonesian')


# ---------------------------- manualy add stopword  ------------------------------------
# append additional stopword
list_stopwords.extend(["yg", "dg", "rt", "dgn", "ny", "d", 'klo', 
                       'kalo', 'amp', 'biar', 'bikin', 'bilang', 
                       'gak', 'ga', 'krn', 'nya', 'nih', 'sih', 
                       'si', 'tau', 'tdk', 'tuh', 'utk', 'ya', 
                       'jd', 'jgn', 'sdh', 'aja', 'n', 't', 
                       'nyg', 'hehe', 'pen', 'u', 'nan', 'loh', 'rt',
                       '&amp', 'yah'])

# ----------------------- add stopword from txt file ------------------------------------
# read txt stopword using pandas
txt_stopword = pd.read_csv("hasil-crawl.csv", names= ["stopwords"], header = None)

# convert stopword string to list & append additional stopword
list_stopwords.extend(txt_stopword["stopwords"][0].split(' '))

# ---------------------------------------------------------------------------------------

# convert list to dictionary
list_stopwords = set(list_stopwords)


#remove stopword pada list token
def stopwords_removal(words):
    return [word for word in words if word not in list_stopwords]

df['abstrak_tokens_WSW'] = df['abstrak_tokens'].apply(stopwords_removal) 


print(df['abstrak_tokens_WSW'].head(20))

0     [haryono, arifin, pengaruh, perilaku, konsumen...
1     [tujuan, penelitian, mengidentifikasi, variabe...
2     [tujuan, penelitian, mendeskripsikan, inovasi,...
3     [tujuan, penelitian, pengaruh, variabel, retai...
4     [hasil, penelitian, perhitungan, credit, risk,...
5     [aththaariq, pengaruh, kompetensi, dosen, kine...
6     [dharma, abidin, syahkesimpulan, pengaruh, sig...
7     [penelitian, rumusan, pelaporan, biaya, kualit...
8     [pln, badan, usaha, milik, negara, bergerak, b...
9     [tujuan, penelitian, mengidentifikasi, variabe...
10    [pendekatan, penelitian, penelitian, pendekata...
11    [diana, agustini, dewi, analisis, kinerja, keu...
12    [tujuan, penelitian, pengaruh, variabel, relat...
13    [muhammad, sholeh, pengaruh, iklan, televisi, ...
14    [tujuan, penelitian, pengaruh, variabel, self,...
15    [satiyah, pengaruh, faktorfaktor, pelatihan, p...
16    [tujuan, penelitian, persepsi, brand, associat...
17    [aplikasi, nyata, pemanfaatan, teknologi, 

# Normalization

Kemudian Normalisasikan dokumen menggunakan fungsi normalized_term yang dibuat dengan bantuan Library Dictionary

In [12]:
normalizad_word = pd.read_csv("hasil-crawl.csv")

normalizad_word_dict = {}

for index, row in normalizad_word.iterrows():
    if row[0] not in normalizad_word_dict:
        normalizad_word_dict[row[0]] = row[1] 

def normalized_term(document):
    return [normalizad_word_dict[term] if term in normalizad_word_dict else term for term in document]

df['abstrak_normalized'] = df['abstrak_tokens_WSW'].apply(normalized_term)

df['abstrak_normalized'].head(20)

0     [haryono, arifin, pengaruh, perilaku, konsumen...
1     [tujuan, penelitian, mengidentifikasi, variabe...
2     [tujuan, penelitian, mendeskripsikan, inovasi,...
3     [tujuan, penelitian, pengaruh, variabel, retai...
4     [hasil, penelitian, perhitungan, credit, risk,...
5     [aththaariq, pengaruh, kompetensi, dosen, kine...
6     [dharma, abidin, syahkesimpulan, pengaruh, sig...
7     [penelitian, rumusan, pelaporan, biaya, kualit...
8     [pln, badan, usaha, milik, negara, bergerak, b...
9     [tujuan, penelitian, mengidentifikasi, variabe...
10    [pendekatan, penelitian, penelitian, pendekata...
11    [diana, agustini, dewi, analisis, kinerja, keu...
12    [tujuan, penelitian, pengaruh, variabel, relat...
13    [muhammad, sholeh, pengaruh, iklan, televisi, ...
14    [tujuan, penelitian, pengaruh, variabel, self,...
15    [satiyah, pengaruh, faktorfaktor, pelatihan, p...
16    [tujuan, penelitian, persepsi, brand, associat...
17    [aplikasi, nyata, pemanfaatan, teknologi, 

# Stemming

Setelah itu lakukan Stemming data mengguanakan Library sastrawi

In [13]:

# import Sastrawi package
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import swifter


# create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# stemmed
def stemmed_wrapper(term):
    return stemmer.stem(term)

term_dict = {}

for document in df['abstrak_normalized']:
    for term in document:
        if term not in term_dict:
            term_dict[term] = ' '
            
print(len(term_dict))
print("------------------------")

for term in term_dict:
    term_dict[term] = stemmed_wrapper(term)
    print(term,":" ,term_dict[term])
    
print(term_dict)
print("------------------------")


# apply stemmed term to dataframe
def get_stemmed_term(document):
    return [term_dict[term] for term in document]

df['abstrak_tokens_stemmed'] = df['abstrak_normalized'].swifter.apply(get_stemmed_term)
print(df['abstrak_tokens_stemmed'])

1132
------------------------
haryono : haryono
arifin : arifin
pengaruh : pengaruh
perilaku : perilaku
konsumen : konsumen
keputusan : putus
pembelian : beli
produk : produk
honda : honda
merek : merek
vario : vario
studi : studi
dealler : dealler
surya : surya
agung : agung
motor : motor
bangkalan : bangkal
tujuan : tuju
penelitian : teliti
dealer : dealer
variabel : variabel
manakah : mana
kebudayaan : budaya
sosial : sosial
pribadi : pribadi
psikologis : psikologis
dominan : dominan
pendekatan : dekat
kuantitatif : kuantitatif
pengamatan : amat
langsung : langsung
responden : responden
penyebaran : sebar
kuesioner : kuesioner
dianalisa : dianalisa
populasi : populasi
dijadikan : jadi
dianalisis : analis
spss : spss
versi : versi
kesimpulan : simpul
pengujian : uji
hipotesis : hipotesis
berpengaruh : pengaruh
simultan : simultan
kunci : kunci
mengidentifikasi : identifikasi
variabelvariabel : variabelvariabel
mempengaruhi : pengaruh
kepuasan : puas
parsial : parsial
metode : metode


Pandas Apply: 100%|██████████| 49/49 [00:00<00:00, 48632.49it/s]

0     [haryono, arifin, pengaruh, perilaku, konsumen...
1     [tuju, teliti, identifikasi, variabelvariabel,...
2     [tuju, teliti, deskripsi, inovasi, unggul, sai...
3     [tuju, teliti, pengaruh, variabel, retailing, ...
4     [hasil, teliti, hitung, credit, risk, ratio, t...
5     [aththaariq, pengaruh, kompetensi, dosen, kerj...
6     [dharma, abidin, syahkesimpulan, pengaruh, sig...
7     [teliti, rumus, lapor, biaya, kualitas, minima...
8     [pln, badan, usaha, milik, negara, gerak, bida...
9     [tuju, teliti, identifikasi, variabelvariabel,...
10    [dekat, teliti, teliti, dekat, kuantitatif, po...
11    [ana, agustini, dewi, analisis, kerja, uang, u...
12    [tuju, teliti, pengaruh, variabel, relationshi...
13    [muhammad, sholeh, pengaruh, iklan, televisi, ...
14    [tuju, teliti, pengaruh, variabel, self, effic...
15    [satiyah, pengaruh, faktorfaktor, latih, kemba...
16    [tuju, teliti, persepsi, brand, association, l...
17    [aplikasi, nyata, manfaat, teknologi, info

Kemudian Rubah hasil data menjadi csv dan melanjutkannya ke proses pengolahan data yang selanjutnya

In [12]:
#df.to_csv("Hasil_Text_Preprocessing.csv")

# Prepare Data

Masukan data yang telah dipreprosessing tadi.

In [14]:
import pandas as pd 
import numpy as np

df = pd.read_csv("Hasil_Text_Preprocessing.csv", usecols=["abstrak_tokens_stemmed"])
df.columns = ["abstrak"]

df.head(20)

,abstrak
0,"['haryono', 'arifin', 'pengaruh', 'perilaku', ..."
1,"['tuju', 'teliti', 'identifikasi', 'variabelva..."
2,"['tuju', 'teliti', 'deskripsi', 'inovasi', 'un..."
3,"['tuju', 'teliti', 'pengaruh', 'variabel', 're..."
4,"['hasil', 'teliti', 'hitung', 'credit', 'risk'..."
5,"['aththaariq', 'pengaruh', 'kompetensi', 'dose..."
6,"['dharma', 'abidin', 'syahkesimpulan', 'pengar..."
7,"['teliti', 'rumus', 'lapor', 'biaya', 'kualita..."
8,"['pln', 'badan', 'usaha', 'milik', 'negara', '..."
9,"['tuju', 'teliti', 'identifikasi', 'variabelva..."


# Term Frekuensi

Hitung Frekuensi Term yang muncul dalam 1 dokumen ini dengan counVectorizer.

In [15]:
from sklearn.feature_extraction.text import CountVectorizer

a=len(document)
document = df['abstrak']

# Create a Vectorizer Object
vectorizer = CountVectorizer()

vectorizer.fit(document)

# Printing the identified Unique words along with their indices
print("Vocabulary: ", vectorizer.vocabulary_)

# Encode the Document
vector = vectorizer.transform(document)

# Summarizing the Encoded Texts
print("Encoded Document is:")
print(vector.toarray())

Vocabulary:  {'haryono': 302, 'arifin': 55, 'pengaruh': 653, 'perilaku': 665, 'konsumen': 438, 'putus': 735, 'beli': 101, 'produk': 706, 'honda': 316, 'merek': 549, 'vario': 983, 'studi': 872, 'dealler': 167, 'surya': 886, 'agung': 11, 'motor': 574, 'bangkal': 82, 'tuju': 950, 'teliti': 909, 'dealer': 166, 'variabel': 979, 'mana': 520, 'budaya': 124, 'sosial': 850, 'pribadi': 700, 'psikologis': 718, 'dominan': 202, 'dekat': 170, 'kuantitatif': 454, 'amat': 36, 'langsung': 480, 'responden': 766, 'sebar': 803, 'kuesioner': 458, 'dianalisa': 187, 'populasi': 682, 'jadi': 364, 'analis': 40, 'spss': 856, 'versi': 985, 'simpul': 836, 'uji': 961, 'hipotesis': 311, 'simultan': 837, 'kunci': 461, 'identifikasi': 319, 'variabelvariabel': 980, 'puas': 721, 'parsial': 636, 'metode': 551, 'depot': 173, 'barokah': 91, 'prambon': 694, 'layan': 486, 'sampel': 793, 'teknik': 904, 'accidental': 2, 'sampling': 795, 'analisis': 41, 'regresi': 753, 'linier': 495, 'ganda': 275, 'hasil': 303, 'independen': 3

In [16]:
a = vectorizer.get_feature_names()

# TF-IDF

## Ekstraksi fitur dan membuat Document Term Matrix (DTM)

Dalam perhitungan LSA (Latent Semantic Analysis) data yang diperlukan hanya TF-IDF. Sehingga pada program ini tidak perlu mencari nilai TF dari dokumen. Untuk mengetahui nilai TF-IDF dapat dilakukan dengan membuat objek dari kelas TfidfVectorizer yang disediakan library scikit-learn.

Rumus Term Frequency (TF):

$$
tf(t,d) = { f_{ t,d } \over \sum_{t' \in d } f_{t,d}}
$$

$ f_{ t,d } \quad\quad\quad\quad$: Jumlah kata t muncul dalam dokumen

$ \sum_{t' \in d } f_{t,d} \quad\quad$: Jumlah seluruh kata yang ada dalam dokumen

Rumus Inverse Document Frequency (IDF):

$$
idf( t,D ) = log { N \over { | \{ d \in D:t \in d \} | } }
$$

$ N \quad\quad\quad\quad\quad$ : Jumlah seluruh dokumen

$ | \{ d \in D:t \in d \} | $ : Jumlah dokumen yang mengandung kata $ t $

Rumus TF - IDF:

$$
tfidf( t,d,D ) = tf( t,d ) \times idf( t,D )
$$

rubah bentuk tf - idf nya menjadi array agar bisa dimasukan kedalam dataframe.

In [17]:
tfidf = TfidfTransformer(use_idf=True, norm='l2', smooth_idf=True)
tf = tfidf.fit_transform(vectorizer.fit_transform(document)).toarray()

lalu masukkan array tadi menjadi sebuah tabel dengan bantuan pandas dataframe.

In [18]:
dfb = pd.DataFrame(data=tf,index=list(range(1, len(tf[:,1])+1, )),columns=[a])
dfb

,abidin,abtrak,accidental,acer,action,adaro,adira,adjusted,administrasi,adnya,...,wildansemsi,windows,wispandonosems,with,xl,xterhadap,yakin,yamaha,yustina,zscore
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.075176,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.058820,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.054291,0.000000,0.054291,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,0.044323,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
10,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


Kemudian convert hasil TF-IDF tadi menjadi csv untuk diproses lebih lanjut.

In [18]:
#dfb.to_csv("Hasil_TF-IDF.csv")